In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import os
import glob
import requests
import ftplib
import io
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO, BytesIO
from zipfile import ZipFile

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [2]:
url = "https://gitlab.com/rcmip/rcmip/-/raw/master/data/protocol/rcmip-concentrations-annual-means-v4-0-0.csv"
req = requests.get(url, headers=header)
data = StringIO(req.text)
RCMIP_concs = pd.read_csv(data).set_index(['Model','Scenario','Region','Variable','Unit','Activity_Id','Mip_Era']).apply(pd.to_numeric).loc[(slice(None),slice(None),'World'),:].droplevel([0,2,5,6])

In [3]:
url = "https://gitlab.com/rcmip/rcmip/-/raw/master/data/protocol/rcmip-emissions-annual-means-v4-0-0.csv"
req = requests.get(url, headers=header)
data = StringIO(req.text)
RCMIP_emms = pd.read_csv(data).set_index(['Model','Scenario','Region','Variable','Unit','Activity_Id','Mip_Era']).apply(pd.to_numeric).loc[(slice(None),slice(None),'World'),:].droplevel([0,2,5,6])

In [4]:
url = "https://gitlab.com/rcmip/rcmip/-/raw/master/data/protocol/rcmip-radiative-forcing-annual-means-v4-0-0.csv"
req = requests.get(url, headers=header)
data = StringIO(req.text)
RCMIP_forc = pd.read_csv(data).set_index(['Model','Scenario','Region','Variable','Unit','Activity_Id','Mip_Era']).apply(pd.to_numeric).loc[(slice(None),slice(None),'World'),:].droplevel([0,2,4,5,6])

In [5]:
# Get GCP CO2 emission estimates
GCP_historical_emissions=pd.read_csv('https://raw.githubusercontent.com/openclimatedata/global-carbon-budget/master/data/historical-budget.csv',index_col=0,usecols=[0,1,2])
# set 1750 value to 0
GCP_historical_emissions.loc[1750,'Fossil-Fuel-Industry'] = 0
# linearly interpolate between 0 and first land-use-change datapoint
GCP_historical_emissions.loc[1750:1849,'Land-Use-Change'] = np.linspace(0,0.99,100) * GCP_historical_emissions.loc[1850,'Land-Use-Change']

In [6]:
RCMIP_forc_out = RCMIP_forc.reset_index().T

In [7]:
RCMIP_forc_out.loc['Variable'] = ['|'.join(x.split('|')[1:]) for x in RCMIP_forc_out.loc['Variable']]

In [8]:
RCMIP_forc_out.to_csv('./rcmip-radiative-forcing-annual-means-v4-0-0_EFaIR.csv')

In [9]:
## map new GIR parameters to old names

RCMIP_emms_name = ['Emissions|F-Gases|PFC|C2F6', 'Emissions|F-Gases|PFC|C3F8', 'Emissions|F-Gases|PFC|C4F10', 'Emissions|F-Gases|PFC|C5F12', 'Emissions|F-Gases|PFC|C6F14', 'Emissions|F-Gases|PFC|C7F16'\
                  , 'Emissions|F-Gases|PFC|C8F18', 'Emissions|F-Gases|PFC|cC4F8'\
                  ,'Emissions|CO2', 'Emissions|Montreal Gases|CCl4', 'Emissions|F-Gases|PFC|CF4','Emissions|Montreal Gases|CFC|CFC113','Emissions|Montreal Gases|CFC|CFC114','Emissions|Montreal Gases|CFC|CFC115'
                  ,'Emissions|Montreal Gases|CFC|CFC11','Emissions|Montreal Gases|CFC|CFC12','Emissions|Montreal Gases|CH2Cl2','Emissions|Montreal Gases|CH3CCl3'\
                  ,'Emissions|Montreal Gases|CHCl3'\
                  ,'Emissions|Montreal Gases|Halon1211','Emissions|Montreal Gases|Halon1301','Emissions|Montreal Gases|Halon2402','Emissions|Montreal Gases|Halon1202'\
                  ,'Emissions|Montreal Gases|HCFC141b','Emissions|Montreal Gases|HCFC142b','Emissions|Montreal Gases|HCFC22'\
                  ,'Emissions|F-Gases|HFC|HFC125','Emissions|F-Gases|HFC|HFC134a','Emissions|F-Gases|HFC|HFC143a','Emissions|F-Gases|HFC|HFC152a','Emissions|F-Gases|HFC|HFC227ea'\
                  ,'Emissions|F-Gases|HFC|HFC236fa','Emissions|F-Gases|HFC|HFC23','Emissions|F-Gases|HFC|HFC245fa','Emissions|F-Gases|HFC|HFC32','Emissions|F-Gases|HFC|HFC365mfc','Emissions|F-Gases|HFC|HFC4310mee'\
                  ,'Emissions|CH4','Emissions|Montreal Gases|CH3Br','Emissions|Montreal Gases|CH3Cl','Emissions|F-Gases|NF3','Emissions|N2O','Emissions|F-Gases|SF6'\
                  ,'Emissions|F-Gases|SO2F2','Emissions|Sulfur','Emissions|NOx','Emissions|CO','Emissions|VOC','Emissions|BC','Emissions|NH3','Emissions|OC','Emissions|NOx|MAGICC Fossil and Industrial|Aircraft']

GIR_name = ['c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8',\
            'carbon_dioxide', 'carbon_tetrachloride', 'cf4', 'cfc113', 'cfc114', 'cfc115', 'cfc11', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3',\
            'halon1211', 'halon1301', 'halon2402', 'halon1202', 'hcfc141b', 'hcfc142b', 'hcfc22',\
            'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc236fa', 'hfc23', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee',\
            'methane', 'methyl_bromide', 'methyl_chloride', 'nf3', 'nitrous_oxide', 'sf6', 'so2f2', 'so2', 'nox', 'co', 'nmvoc','bc','nh3','oc','nox_avi']

RCMIP_to_GIR_map_emms = dict(zip(RCMIP_emms_name,GIR_name))

RCMIP_to_GIR_map_concs = dict(zip(['Atmospheric Concentrations|'+'|'.join(x.split('|')[1:]) for x in RCMIP_to_GIR_map_emms.keys()],GIR_name))

GIR_to_RCMIP_map = pd.DataFrame(index=GIR_name)
GIR_to_RCMIP_map['native_emms_unit'] = 'Mt'
GIR_to_RCMIP_map.loc['carbon_dioxide','native_emms_unit'] = 'GtC'
GIR_to_RCMIP_map.loc['nitrous_oxide','native_emms_unit'] = 'MtN2O-N2'
GIR_to_RCMIP_map.loc[['nox','nox_avi'],'native_emms_unit'] = 'MtNO2'

GIR_to_RCMIP_map['native_concs_unit'] = 'ppb'
GIR_to_RCMIP_map.loc['carbon_dioxide','native_concs_unit'] = 'ppm'
GIR_to_RCMIP_map.loc[['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi'],'native_concs_unit'] = 'Mt'
GIR_to_RCMIP_map.loc[['nox','nox_avi'],'native_concs_unit'] = 'MtNO2'

GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_emms.values(),'RCMIP_emms_key'] = list(RCMIP_to_GIR_map_emms.keys())
GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_emms.values(),'RCMIP_emms_unit'] = RCMIP_emms.loc['ssp245'].reset_index().set_index('Variable').iloc[:,0].loc[GIR_to_RCMIP_map['RCMIP_emms_key'].values].values
GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_emms.values(),'RCMIP_emms_scaling'] = 1/1000
GIR_to_RCMIP_map.loc[['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi'],'RCMIP_emms_scaling'] = 1
GIR_to_RCMIP_map.loc['nitrous_oxide','RCMIP_emms_scaling'] = 28/(44*1000)
GIR_to_RCMIP_map.loc['methane','RCMIP_emms_scaling'] = 1
GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_emms_scaling'] = 12/(44.01*1000)

GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_key'] = list(RCMIP_to_GIR_map_concs.keys())
GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_unit'] = RCMIP_concs.loc['ssp245'].reset_index().set_index('Variable').iloc[:,0].reindex(GIR_to_RCMIP_map['RCMIP_concs_key'].values).values
GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_scaling'] = 1/1000
GIR_to_RCMIP_map.loc['nitrous_oxide','RCMIP_concs_scaling'] = 1
GIR_to_RCMIP_map.loc['methane','RCMIP_concs_scaling'] = 1
GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_concs_scaling'] = 1
GIR_to_RCMIP_map.loc[['so2','nox','co','nmvoc','bc','nh3','oc','nox_avi'],'RCMIP_concs_scaling'] = np.nan

In [10]:
def RCMIP_to_GIR_input_emms(scenario):
    _out = RCMIP_emms.loc[scenario].loc[GIR_to_RCMIP_map.RCMIP_emms_key.tolist()].droplevel([1])
    _out_index = _out.index.map({v: k for k, v in GIR_to_RCMIP_map.RCMIP_emms_key.to_dict().items()})
    _out.set_index(_out_index,inplace=True)
    _out = _out.T.mul(GIR_to_RCMIP_map.RCMIP_emms_scaling)
    _out.index = _out.index.astype(int)
    _out.columns.name='Variable'
    return _out.apply(pd.to_numeric)

In [11]:
RCMIP_emms_out = pd.concat([RCMIP_to_GIR_input_emms(x).interpolate(limit_area='inside') for x in RCMIP_emms.index.levels[0]],axis=1,keys=RCMIP_emms.index.levels[0]).T.reset_index().T

In [12]:
for scenario in RCMIP_emms_out.loc['Scenario'].unique():
    if 'ssp' in scenario or 'rcp' in scenario or 'historical' in scenario:
        ## remove pre-industrial emission values:
        if 'ssp' in scenario:
            RCMIP_emms_out.loc[range(1750,2501),RCMIP_emms_out.loc['Scenario']==scenario] -= RCMIP_emms_out.loc[1750,RCMIP_emms_out.loc['Scenario']==scenario]
        if 'rcp' in scenario:
            RCMIP_emms_out.loc[range(1765,2501),RCMIP_emms_out.loc['Scenario']==scenario] -= RCMIP_emms_out.loc[1765,RCMIP_emms_out.loc['Scenario']==scenario]
        if scenario == 'historical':
            RCMIP_emms_out.loc[range(1750,2501),RCMIP_emms_out.loc['Scenario']==scenario] -= RCMIP_emms_out.loc[1750,RCMIP_emms_out.loc['Scenario']==scenario]
        if scenario == 'historical-cmip5':
            RCMIP_emms_out.loc[range(1765,2501),RCMIP_emms_out.loc['Scenario']==scenario] -= RCMIP_emms_out.loc[1765,RCMIP_emms_out.loc['Scenario']==scenario]
            
        RCMIP_emms_out.loc[range(1750,2015),(RCMIP_emms_out.loc['Scenario']==scenario)&(RCMIP_emms_out.loc['Variable']=='carbon_dioxide')] = GCP_historical_emissions.sum(axis=1).loc[1750:2014]
        sf = GCP_historical_emissions.sum(axis=1).loc[2015] / RCMIP_emms_out.loc[2015,(RCMIP_emms_out.loc['Scenario']==scenario)&(RCMIP_emms_out.loc['Variable']=='carbon_dioxide')]
        RCMIP_emms_out.loc[range(2015,2026),(RCMIP_emms_out.loc['Scenario']==scenario)&(RCMIP_emms_out.loc['Variable']=='carbon_dioxide')] *= np.linspace(sf,1,11)

In [36]:
RCMIP_emms_out.T.set_index(['Scenario','Variable']).dropna(how='all').reset_index().T.to_csv('rcmip-emissions-annual-means-v4-0-0_EFaIR.csv')

In [17]:
def RCMIP_to_GIR_input_concs(scenario):
    _out = RCMIP_concs.loc[scenario].loc[GIR_to_RCMIP_map.RCMIP_concs_key.tolist()].droplevel([1])
    _out_index = _out.index.map({v: k for k, v in GIR_to_RCMIP_map.RCMIP_concs_key.to_dict().items()})
    _out.set_index(_out_index,inplace=True)
    _out = _out.T.mul(GIR_to_RCMIP_map.RCMIP_concs_scaling)
    _out.index = _out.index.astype(int)
    _out.columns.name='Variable'
    return _out.apply(pd.to_numeric)

In [19]:
RCMIP_concs_out = pd.concat([RCMIP_to_GIR_input_concs(x).interpolate(limit_area='inside') for x in RCMIP_concs.index.levels[0]],axis=1,keys=RCMIP_concs.index.levels[0]).T.reset_index().T

In [32]:
RCMIP_concs_out.T.set_index(['Scenario','Variable']).dropna(how='all').reset_index().T.to_csv('rcmip-concentrations-annual-means-v4-0-0_EFaIR.csv')